![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [12]:
import pandas as pd
import numpy as np

# Start coding here


#read csv file as a data
data = pd.read_csv("bank_marketing.csv")

#show first five row by head()
print(data.head())
#check null and types in each column
print(data.info())
#check 
print(data.describe())

   client_id  age        job  ... euribor3m nr_employed   y
0          0   56  housemaid  ...     4.857      5191.0  no
1          1   57   services  ...     4.857      5191.0  no
2          2   37   services  ...     4.857      5191.0  no
3          3   40     admin.  ...     4.857      5191.0  no
4          4   56   services  ...     4.857      5191.0  no

[5 rows x 22 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month   

In [13]:
import pandas as pd
import numpy as np

# Split data into client, campaign, and economics
# Create a client dataset
client_data = data[["client_id","age","job", "marital", "education","credit_default","housing","loan"]]
# Print first five rows of client data
#print(client_data.head())

# Create a campaign dataset
campaign_data = data[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
# Print first five rows of campaign dataset
#print(campaign_data.head())

# Create an economics dataset
economics_data = data[["client_id","emp_var_rate","cons_price_idx","euribor3m","nr_employed"]]
# Print first five rows of economics dataset
#print(economics_data.head())

# Rename in each dataset
# Rename client_id to id in client dataset
client_data.rename(columns={"client_id": "id"}, inplace=True)

# Rename "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign dataset
campaign_data.rename(columns={"duration": "contact_duration",
                              "previous": "previous_campaign_contacts",
                              "y": "campaign_outcome",
                              "poutcome": "previous_outcome",
                              "campaign": "number_contacts"}, inplace=True)

# Rename "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics dataset
economics_data.rename(columns={"euribor3m": "euribor_three_months",
                               "nr_employed": "number_employed"}, inplace=True)

# Clean education column
client_data["education"] = client_data["education"].str.replace(".", "_")
client_data["education"] = client_data["education"].replace("unknown", np.NaN)

# Clean job column
client_data["job"] = client_data["job"].str.replace(",", "")

# Change poutcome column
campaign_data["previous_outcome"] = campaign_data["previous_outcome"].replace("nonexistent", np.NaN)
campaign_data["previous_outcome"] = campaign_data["previous_outcome"].map({"success": 1, "failure": 0})

# Change campaign_outcome column
campaign_data["campaign_outcome"] = campaign_data["campaign_outcome"].map({"success": 1, "failure": 0})

# Add campaign_id column
campaign_data["campaign_id"] = 1

# Capitalize month and day and convert to datetime
campaign_data["month"] = campaign_data["month"].str.capitalize()
campaign_data["year"] = "2022"
campaign_data["day"] = campaign_data["day"].astype(str)
campaign_data["last_contact_date"] = campaign_data["year"] + "-" + campaign_data["month"] + "-" + campaign_data["day"]
campaign_data["last_contact_date"] = pd.to_datetime(campaign_data["last_contact_date"], format="%Y-%b-%d")
campaign_data.drop(columns=["month", "day", "year"], inplace=True)

In [14]:
# Save table to CSV files
client_data.to_csv("client.csv", index = False)
campaign_data.to_csv("campaign", index = False)
economics_data.to_csv("economics.csv", index = False)

client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""
